In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager as fm
import matplotlib
import timeit
import feather
%matplotlib inline

In [2]:
res = pd.read_feather('DBus_data_first_clean.feather')

In [3]:
res.shape

(70904023, 11)

In [4]:
# Attaching leading zeroes to VehicleJourneyIDs and StopIDs of less than 4 digits
# code from: https://stackoverflow.com/questions/33243763/pandas-add-leading-0-to-string-values-so-all-values-are-equal-len

res.VehicleJourneyID = res.VehicleJourneyID.apply(lambda x: str(int(x)).zfill(4))
res.StopID = res.StopID.apply(lambda x: str(int(x)).zfill(4))

In [5]:
res.head(4)

,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,Delay,VehicleID,StopID,AtStop,Day
0,2013-01-31 00:00:01,00131005,2013-01-30,3406,-6.273923,53.343307,-235,33608,1998,False,Wednesday
1,2013-01-31 00:00:03,00071003,2013-01-30,6069,-6.230700,53.317432,0,43003,0008,True,Wednesday
2,2013-01-31 00:00:03,00671001,2013-01-30,0905,-6.325533,53.346302,-396,33438,2248,False,Wednesday
3,2013-01-31 00:00:03,00831001,2013-01-30,5613,-6.284449,53.420197,-490,40012,1552,False,Wednesday


### Part I. Replace StopID that not exist in googletransit data to stop_id that is close to it.
Origin: Delete stopID not in that trip_id (same meaning as JourneyPatternID) Revised:0805

0805 revised: 
First, find the possible stop location by calaulate maximum occurance AtStop=1 from gps data.
Second, use stopID as key to calculate distance with googletransit data.
=> Find three situation: 
(1) Google transit data exists duplicate stopID with different lon/lat.
(2) There are ? number of stopID exist in gps data but not in google transit data.
(3) There are 60 number of stopID location is over 200 meters difference from google transit data.

#### Step 1. Prepare google transit stop data
Due to there are duplicate stop short name (last four digits of stop_id) in google transit data. We compare the location to gps data, we found out other than stop_id has 'VIR' string in there should delete. 8250DB00661, 8250DB002561, 8220PB003854, 8220DB007270 also should delete. (The same process should apply in database)

In [7]:
# First, clean google transit stop location data.
stops_2012 = pd.read_csv('stops.txt')
stops_2012.head(5)

,stop_id,stop_name,stop_lat,stop_lon
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693
1,8220DB000003,"Parnell Square, Rotunda",53.352308,-6.263781
2,8220DB000004,"Parnell Square, Rotunda",53.352565,-6.264161
3,8220DB000006,"Parnell Square, Rotunda",53.352740,-6.264439
4,8220DB000007,"Parnell Square, Rotunda",53.352832,-6.264556


In [8]:
stops_2012 = stops_2012[stops_2012.stop_id.str.find('VIR') == -1]
stops_2012['StopID'] = stops_2012['stop_id'].str[-4:]
# Revised: 
# There is possibility of those duplicate shows as other StopID in gpd data, so here is to find out.
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8250DB00661')].index, col='StopID', value='0661_1')
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8250DB002561')].index, col='StopID', value='2561_1')
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8220PB003854')].index, col='StopID', value='3854_1')
stops_2012.set_value(index = stops_2012[(stops_2012.stop_id == '8220DB007270')].index, col='StopID', value='7270_1')
stops_2012.head(5)

,stop_id,stop_name,stop_lat,stop_lon,StopID
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002
1,8220DB000003,"Parnell Square, Rotunda",53.352308,-6.263781,0003
2,8220DB000004,"Parnell Square, Rotunda",53.352565,-6.264161,0004
3,8220DB000006,"Parnell Square, Rotunda",53.352740,-6.264439,0006
4,8220DB000007,"Parnell Square, Rotunda",53.352832,-6.264556,0007


In [9]:
# Make sure no duplicates.
stops_2012[stops_2012['StopID'].duplicated()]

,stop_id,stop_name,stop_lat,stop_lon,StopID


#### Step 2. Find possible position of StopID in gps data

In [9]:
res_temp = res
res_temp["JourneyPatternID"] = res_temp["JourneyPatternID"].astype("category")
res_temp = res_temp[res_temp.AtStop == True]

* Longtitude

In [10]:
# First groupby StopID with Lon and found the max count

gb = res_temp.groupby(['StopID','Lon'])
gbLon = pd.DataFrame(gb['AtStop'].count())
gbLon.reset_index(['StopID','Lon'], inplace=True)

In [11]:
# Find the maximun occurance Lon for each StopID

maxCountLon = gbLon[gbLon.groupby(['StopID'])['AtStop'].transform('max') == gbLon['AtStop']]
maxCountLon.shape

(4733, 3)

* Latitude

In [12]:
# First groupby StopID with Lon and found the max count

gb = res_temp.groupby(['StopID','Lat'])
gbLat = pd.DataFrame(gb['AtStop'].count())
gbLat.reset_index(['StopID','Lat'], inplace=True)

In [13]:
# Find the maximun occurance Lat for each StopID

maxCountLat = gbLat[gbLat.groupby(['StopID'])['AtStop'].transform('max') == gbLat['AtStop']]
maxCountLat.shape

(4735, 3)

* Lon/Lat

In [14]:
# Merge by StopID

maxCountLonLat = pd.merge(maxCountLon, maxCountLat, on=['StopID'], how='inner')
maxCountLonLat.shape

(4925, 5)

In [15]:
# There are 8 StopID cannot find the obvious Co-ordinate, use average

gb = maxCountLonLat.groupby('StopID')
gb = gb.mean()
gb.reset_index(['StopID'], inplace=True)

# Don't know why there are some StopID not exist appear in the index

gb.dropna(axis=0, how='any', inplace=True)
gb.drop(['AtStop_x', 'AtStop_y'], axis=1, inplace=True)
gps_est_lat_lon = gb.loc[:, ['StopID', 'Lat', 'Lon']]

In [16]:
# Now we have the possible lat/lon of each StopID
gps_est_lat_lon.head(3)

,StopID,Lat,Lon
0,0010,53.353436,-6.265345
1,0100,53.303043,-6.322296
2,1000,53.390884,-6.201829


#### Step 3. For those StopID exist in gps data not in googletransit data, find the replacement StopID

In [17]:
# Merge google_transit data with possible lat/lon of gps data
compare_google12_gpsest = pd.merge(gps_est_lat_lon, stops_2012, on='StopID', how='outer')
compare_google12_gpsest.head(2)

,StopID,Lat,Lon,stop_id,stop_name,stop_lat,stop_lon
0,0010,53.353436,-6.265345,8220DB000010,"Parnell Square, Rotunda",53.353383,-6.265389
1,0100,53.303043,-6.322296,8230DB000100,"Wellington Lane, Templeogue",53.303083,-6.322361


In [18]:
# Find StopID exist in gps data not in google transit data
not_in_google12 = compare_google12_gpsest[compare_google12_gpsest.stop_name.isnull()]
not_in_google12.drop(['stop_id', 'stop_name', 'stop_lat', 'stop_lon'], axis=1, inplace=True)
not_in_google12.head(3)

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,StopID,Lat,Lon
2337,3581,53.446697,-6.171781
2338,3582,53.448593,-6.167941
2392,3637,53.448624,-6.167745


In [19]:
# In order to find the possible replacement here to product the "not_in_google12" with "stops_2012"
stops_2012_temp = stops_2012
stops_2012_temp['merge_id'] = np.ones((stops_2012_temp.shape[0],1 ), dtype=np.int16)
not_in_google12['merge_id'] = np.ones((not_in_google12.shape[0],1 ), dtype=np.int16)
production = pd.merge(stops_2012_temp, not_in_google12, on='merge_id', suffixes=('_google','_gps'))
production.head(3)

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,stop_id,stop_name,stop_lat,stop_lon,StopID_google,merge_id,StopID_gps,Lat,Lon
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3581,53.446697,-6.171781
1,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3582,53.448593,-6.167941
2,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3637,53.448624,-6.167745


In [20]:
# Here calculate distance between each production.
from geopy.distance import great_circle
production['distance_diff'] = production.loc[:, ['Lat','Lon','stop_lat','stop_lon']].apply(lambda x: great_circle((x[0],x[1]), (x[2],x[3])).meters, axis=1)
production.head(2)

,stop_id,stop_name,stop_lat,stop_lon,StopID_google,merge_id,StopID_gps,Lat,Lon,distance_diff
0,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3581,53.446697,-6.171781,12145.871698
1,8220DB000002,"Parnell Square, Rotunda",53.352244,-6.263693,0002,1,3582,53.448593,-6.167941,12456.518848


In [21]:
# Use the minimal distance as the possible replacement.
production_min = production[production.groupby(['StopID_gps'])['distance_diff'].transform('min') == production['distance_diff']]
production_min.head(2)

,stop_id,stop_name,stop_lat,stop_lon,StopID_google,merge_id,StopID_gps,Lat,Lon,distance_diff
16303,8220DB000339,"Burgh Quay, Dublin City South",53.347255,-6.256896,0339,1,7543,53.347473,-6.255754,79.634691
16305,8220DB000339,"Burgh Quay, Dublin City South",53.347255,-6.256896,0339,1,7545,53.347420,-6.256560,28.881757


#### Observation
From table can find out that, duplicate in googletransit data:
* 8220DB007270 -> is 7220 in gps data
* 8220PB003854,  8250DB00661 can drop
* 8250DB002561 -> is 7550 in gps

In [22]:
# So we will replace 8220DB007270 to '7220' and 8250DB002561 to '7250' in google transit.
# Others replace gps StopID to corresponding StopID_google
# Here drop StopID_google == '3854_1' is because duplicate
production_min = production_min[~production_min.StopID_google.isin(['3854_1', '7270_1', '2561_1'])]

#### Solution
(1) Replace those distance < 150 to google transit stop_id
(2) Delete those distance > 150

In [23]:
replace = production_min[production_min.distance_diff < 150].loc[:,['StopID_gps', 'StopID_google']]
delete = production_min[production_min.distance_diff >= 150].loc[:,['StopID_gps', 'StopID_google']]

In [24]:
delete.head(3)

,StopID_gps,StopID_google
165708,7482,0323
165784,7502,0324
165831,7479,0327


#### Step 4. Replace and delete

In [25]:
# Delete
temp = pd.merge(res, delete, left_on='StopID', right_on='StopID_gps', how='outer')
temp = temp[temp.StopID_gps.isnull()]

In [26]:
temp.drop(['StopID_gps', 'StopID_google'], axis=1, inplace=True)

In [27]:
# Replace
temp2 = pd.merge(temp, replace, left_on='StopID', right_on='StopID_gps', how='outer')
temp2['StopID'] = temp2.loc[:,['StopID_google', 'StopID']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

In [28]:
res = temp2.loc[:,['Timestamp', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Lon', 'Lat', 'VehicleID', 'StopID', 'AtStop', 'Day']]

### Part II. Delete stopID should not exist on that JourneyPatternID
In this part, we use google transit data as base to delete those StopID should not exist on that JPID. In order to do that, we need to find the possible mapping of JourneyPatternID to shape_id in google transit data. First we will find those JPID is certainly can map to a shape_id. Others that cannot not sure map to certaon trip_id will use the other way to check the which StopID should not exist on that JourneyPatternID.

#### Step 1. Prepare data to compare
First, find the stop_id of each trip_id in GTFS data and the StopID of the JourneyPatternID.

#### 1. GTFS

In [29]:
# Read in stop_times.csv and trips.csv
stop_times_2012 = pd.read_csv('stop_times.csv')
trips_2012 = pd.read_csv('trips.csv')

# Merge by trip_id
# trip_id here has the same meaning of JourneyPatternID
merge = pd.merge(stop_times_2012, trips_2012, on='trip_id', how='outer')

# Keep only necessary columns
merge.drop(['arrival_time','departure_time','pickup_type','drop_off_type','service_id','shape_dist_traveled'], axis=1, inplace=True)

In [30]:
# Seporate trip_id to route, stop_id to StopID
# Route
merge['route_short'] = merge['trip_id'].apply(lambda x: x[x.index('-')+1:])
merge['route_short'] = merge['route_short'].apply(lambda x: x[: x.index('-')])
merge['route_short'] = merge['route_short'].apply(lambda x: str(x).zfill(4))

# StopID
merge['StopID'] = merge['stop_id'].apply(lambda x: x[-4:])

In [31]:
# Find StopID and the sequence of that shape_id
gb = merge.groupby(['shape_id', 'route_short', 'direction_id','stop_sequence', 'StopID'])
gbc = gb.count()
gbc.reset_index(['shape_id', 'route_short', 'direction_id','stop_sequence', 'StopID'], inplace=True)
transit_shapeID_stopID = gbc.drop(['trip_id','stop_id','route_id','trip_headsign'], axis=1)

In [32]:
# Now we have shape_id to its corresponding StopID and sequence
transit_shapeID_stopID.head(5)

,shape_id,route_short,direction_id,stop_sequence,StopID
0,0-1-y12-1.1.O,0001,0,1,0226
1,0-1-y12-1.1.O,0001,0,2,0228
2,0-1-y12-1.1.O,0001,0,3,0229
3,0-1-y12-1.1.O,0001,0,4,0227
4,0-1-y12-1.1.O,0001,0,5,0230


#### 2. JourneyPatternID to its possible StopID

In [33]:
# Get JourneyPatternID and its StopID
temp = res[res.AtStop == True]
gb = temp.groupby(['JourneyPatternID', 'StopID'])
gc = gb.count()

In [34]:
# Clean up
gbc = gc[~gc.Timestamp.isnull()] # Doing this here is cause the feather will include all column, so clean up a little here
gbc.reset_index(['JourneyPatternID', 'StopID'], inplace=True)
gps_JPID_StopID = gbc.loc[:,['JourneyPatternID', 'StopID', 'AtStop']]
gps_JPID_StopID = gps_JPID_StopID.rename(columns = {'AtStop':'Count'})

In [35]:
# Separate JourneyPatternID to route and direction
gps_JPID_StopID['route_short'] = gps_JPID_StopID['JourneyPatternID'].str[:4]
gps_JPID_StopID['Direction'] = gps_JPID_StopID['JourneyPatternID'].str[4:]


# Now we have JourneyPatternID to its possible StopID
gps_JPID_StopID.head(5)

,JourneyPatternID,StopID,Count,route_short,Direction
0,00010001,0044,799.0,0001,0001
1,00010001,0045,1398.0,0001,0001
2,00010001,0046,1092.0,0001,0001
3,00010001,0047,1010.0,0001,0001
4,00010001,0048,1005.0,0001,0001


0720 add
#### Step 2. Check if the number of trip_id is match with JourneyPatternID
In this section, we try to find out if the number of trip_id under each route in googletransit2012 is the same number of JourneyPatternID in gps data. For example, for route 007B has trip_id "0-7B-y12-1.176.O" and "0-7B-y12-1.177.I"; JourneyPatternID "007B0001" and "007B1001". So we know at least there is one-to-one mapping. Other scenario see document "compare_route_googletransit_vs_gps.xlsx"

In [36]:
gb_tss = transit_shapeID_stopID.groupby(['route_short','shape_id'])
gb_tss_c = gb_tss.count()
gb_tss_c = gb_tss_c.reset_index(['route_short','shape_id'])
gb_tss_c.drop(['direction_id', 'stop_sequence', 'StopID'], axis=1, inplace=True)
gb_tss_c.head(5)

,route_short,shape_id
0,0001,0-1-y12-1.1.O
1,0001,0-1-y12-1.2.O
2,0001,0-1-y12-1.3.O
3,0001,0-1-y12-1.4.I
4,0001,0-1-y12-1.5.I


In [37]:
gb_gjs = gps_JPID_StopID.groupby(['route_short','JourneyPatternID'])
gb_gjs_c = gb_gjs.count()
gb_gjs_c = gb_gjs_c.reset_index(['route_short','JourneyPatternID'])
gb_gjs_c.drop(['StopID', 'Count', 'Direction'], axis=1, inplace=True)
gb_gjs_c.head(5)

,route_short,JourneyPatternID
0,0001,00010001
1,0001,00010002
2,0001,00010003
3,0001,00011001
4,0001,00011002


In [38]:
# Route exist both in GPS and google transit
inter = set(gb_tss_c.route_short.unique()) & set(gb_gjs_c.route_short.unique())

tss_l = []
gjs_l = []

for r in inter:
    tss_l.append(gb_tss_c[gb_tss_c.route_short == r])
    gjs_l.append(gb_gjs_c[gb_gjs_c.route_short == r])

#     
tss_l[0] = tss_l[0].reset_index()
gjs_l[0] = gjs_l[0].reset_index()
df = pd.concat([tss_l[0], gjs_l[0]], axis=1)


for i in range(1, len(tss_l)):
    tss_l[i] = tss_l[i].reset_index()
    gjs_l[i] = gjs_l[i].reset_index()
    temp = pd.concat([tss_l[i], gjs_l[i]], axis=1)
    df = pd.concat([df, temp], axis=0)


In [39]:
# Route not in GPS data
diff = set(gb_tss_c.route_short.unique()) - inter

df2 = pd.DataFrame(columns = df.columns)
for r in diff:
    x = gb_tss_c[gb_tss_c.route_short == r]
    x = x.reset_index()
    y = gb_gjs_c[gb_gjs_c.route_short == r]
    y = y.reset_index()
    temp = pd.concat([x, y], axis=1)
    df2 = pd.concat([df2, temp], axis=0)

# Route not in google transit data

diff = set(gb_gjs_c.route_short.unique()) - inter

#df2 = pd.DataFrame(columns = df.columns)
for r in diff:
    x = gb_tss_c[gb_tss_c.route_short == r]
    x = x.reset_index()
    y = gb_gjs_c[gb_gjs_c.route_short == r]
    y = y.reset_index()
    temp = pd.concat([x, y], axis=1)
    df2 = pd.concat([df2, temp], axis=0)


In [40]:
out = pd.concat([df,df2], axis=0)
out.drop(['index'], axis=1, inplace=True)
out.to_csv('compare_route_googletransit_vs_gps.csv', index=False)

#### Step3. Mapping the possible trip_id to JourneyPatternID
In order to find the possible trip_id to JourneyPatternID, here transfer the sequence of stop_id in each trip_id to set. The same as StopID of each JourneyPatternID.
The reason to transfer to a set is because we can use the set intersection method to find out the common StopID between two sets. Example, JourneyPatternID 00010001 has stopID {1,2,3,4}, 00010002 has {1,3,5} in gps data and trip_id 0-1-y12-1.1.O has stop_id {1,2,3,4}. So we can get the length of intersection from 00010001 is 4 and 00010002 is 2. Which means 00010001 is more likely is trip_id 0-1-y12-1.1.O

0720 Revised
#### 1. Remain only those route is both exist in googletransit and gps data
Due to the observation above, we found out there are few route not exist in googletransit. So, we will delete it at this moment.

In [41]:
# In diff set are those route not exist in googletransit
inter = set(gb_tss_c.route_short.unique()) & set(gb_gjs_c.route_short.unique())
diff = set(gb_gjs_c.route_short.unique()) - inter

res['Route'] = res['JourneyPatternID'].str[:4]
res = res[~res.Route.isin(list(diff))]

#### 2. Get StopID set of googletransit data

In [42]:
# Sort by shape_id and stopID
transit_shapeID_stopID.sort_values(['shape_id', 'StopID'], inplace=True)

# Store each StopID of each shape into set
shapeID_stopID_set = {}
shape_unique = transit_shapeID_stopID.shape_id.unique()

for s in shape_unique:
    temp = []
    for stopid in transit_shapeID_stopID[transit_shapeID_stopID.shape_id == s]['StopID']:
        temp.append(stopid)
        temp.sort()
    shapeID_stopID_set[s] = [set(temp), temp]

In [43]:
# Here transform to dataframe so can merge later

# Get the shape_id to its stopID list
shapeID_stopID_df = pd.DataFrame(shapeID_stopID_set, index=['StopID_set', 'StopID_list'])
shapeID_stopID_df = shapeID_stopID_df.T
shapeID_stopID_df = shapeID_stopID_df.reset_index()
shapeID_stopID_df = shapeID_stopID_df.rename(columns = {'index':'shape_id'})

In [44]:
# Separate shape_id to route
shapeID_stopID_df['route_short'] = shapeID_stopID_df['shape_id'].apply(lambda x: x[x.index('-')+1:][:x[x.index('-')+1:].index('-')])
shapeID_stopID_df['route_short'] = shapeID_stopID_df['route_short'].apply(lambda x: str(x).zfill(4))

In [45]:
# Now we get the shape_id to its StopID set
shapeID_stopID_df.head(5)

,shape_id,StopID_set,StopID_list,route_short
0,0-1-y12-1.1.O,"{0050, 0271, 0265, 0340, 1641, 0229, 0354, 280...","[0044, 0045, 0046, 0047, 0048, 0049, 0050, 005...",0001
1,0-1-y12-1.2.O,"{0271, 0265, 0340, 0354, 2804, 0378, 0355, 035...","[0265, 0271, 0340, 0350, 0351, 0352, 0353, 035...",0001
2,0-1-y12-1.3.O,"{0050, 0271, 0265, 1641, 0229, 0045, 4432, 022...","[0044, 0045, 0046, 0047, 0048, 0049, 0050, 005...",0001
3,0-1-y12-1.4.I,"{0205, 0223, 0010, 0319, 0017, 0384, 0085, 020...","[0010, 0012, 0014, 0015, 0017, 0018, 0019, 002...",0001
4,0-1-y12-1.5.I,"{0010, 0319, 0384, 0392, 0387, 0396, 0399, 039...","[0010, 0278, 0319, 0371, 0381, 0382, 0383, 038...",0001


#### 3. StopID set of gps data

In [46]:
# The same operation here for gps data
# Sort by JourneyPatternID and stopID
gps_JPID_StopID.sort_values(['JourneyPatternID', 'StopID'], inplace=True)

# Store each StopID of each JourneyPatternID into list
JPID_stopID_set = {}
JPID_unique = gps_JPID_StopID.JourneyPatternID.unique()

for s in JPID_unique:
    temp = []
    for stopid in gps_JPID_StopID[gps_JPID_StopID.JourneyPatternID == s]['StopID']:
        temp.append(stopid)
        temp.sort()
    JPID_stopID_set[s] = [set(temp), temp]

In [47]:
# Get the JourneyPatternID to its stopID list
JPID_stopID_df = pd.DataFrame(JPID_stopID_set, index=['StopID_set', 'StopID_list'])
JPID_stopID_df = JPID_stopID_df.T
JPID_stopID_df = JPID_stopID_df.reset_index()
JPID_stopID_df = JPID_stopID_df.rename(columns = {'index':'JourneyPatternID'})
JPID_stopID_df.head(5)

,JourneyPatternID,StopID_set,StopID_list
0,00010001,"{0050, 0271, 0265, 0340, 1641, 0319, 0229, 035...","[0044, 0045, 0046, 0047, 0048, 0049, 0050, 005..."
1,00010002,"{0271, 0265, 0340, 0354, 2804, 0378, 0355, 035...","[0265, 0271, 0340, 0350, 0351, 0352, 0353, 035..."
2,00010003,"{0205, 0050, 0271, 0265, 1641, 0229, 0045, 443...","[0045, 0046, 0047, 0048, 0049, 0050, 0051, 005..."
3,00011001,"{0205, 0271, 0265, 0340, 0223, 0010, 0319, 001...","[0010, 0012, 0014, 0015, 0017, 0018, 0019, 002..."
4,00011002,"{0265, 0010, 0319, 0384, 0378, 0392, 0387, 039...","[0010, 0265, 0278, 0319, 0371, 0372, 0378, 038..."
5,00040001,"{0408, 0410, 0038, 0186, 6085, 6234, 2051, 040...","[0006, 0037, 0038, 0039, 0040, 0091, 0094, 011..."
6,00040002,"{0038, 0186, 0185, 0147, 0113, 0037, 0332, 018...","[0037, 0038, 0040, 0112, 0113, 0115, 0127, 014..."
7,00041001,"{0281, 0483, 2051, 0148, 0090, 0472, 0322, 009...","[0006, 0027, 0028, 0029, 0037, 0040, 0090, 009..."
8,00041002,"{0281, 0483, 2051, 7330, 0469, 0475, 0488, 608...","[0281, 0400, 0469, 0471, 0473, 0475, 0478, 047..."
9,00070001,"{0408, 3041, 0410, 2046, 3037, 4982, 0409, 204...","[0273, 0402, 0405, 0408, 0409, 0410, 0411, 041..."


In [48]:
# Get the route from JourneyPatternID
JPID_stopID_df['route_short'] = JPID_stopID_df['JourneyPatternID'].str[:4]

In [49]:
# Now we also have JourneyPatternID to its StopID list
JPID_stopID_df.head(5)

,JourneyPatternID,StopID_set,StopID_list,route_short
0,00010001,"{0050, 0271, 0265, 0340, 1641, 0319, 0229, 035...","[0044, 0045, 0046, 0047, 0048, 0049, 0050, 005...",0001
1,00010002,"{0271, 0265, 0340, 0354, 2804, 0378, 0355, 035...","[0265, 0271, 0340, 0350, 0351, 0352, 0353, 035...",0001
2,00010003,"{0205, 0050, 0271, 0265, 1641, 0229, 0045, 443...","[0045, 0046, 0047, 0048, 0049, 0050, 0051, 005...",0001
3,00011001,"{0205, 0271, 0265, 0340, 0223, 0010, 0319, 001...","[0010, 0012, 0014, 0015, 0017, 0018, 0019, 002...",0001
4,00011002,"{0265, 0010, 0319, 0384, 0378, 0392, 0387, 039...","[0010, 0265, 0278, 0319, 0371, 0372, 0378, 038...",0001


#### 3. Now merge and find the length of common StopID between two set

In [50]:
# Merge two datafram base on the route_short (cause for now we still don't know which JourneyPatternID is which trip_id)
merge = pd.merge(shapeID_stopID_df, JPID_stopID_df, on='route_short', how='inner')

In [51]:
# Find the common StopID of googletransport and gps
# (x[0] & x[1]) is for python set structure to find the common elements between two sets
merge['common_len'] = merge.loc[:,['StopID_set_x', 'StopID_set_y']].apply(lambda x: len(x[0] & x[1]), axis=1)
merge['shape_id_len'] = merge['StopID_list_x'].apply(lambda x: len(x))
merge['JourneyPatternID_len'] = merge['StopID_list_y'].apply(lambda x: len(x))

merge['JPID_len-shapid_len'] = merge['JourneyPatternID_len'] - merge['shape_id_len']
merge = merge[merge['JPID_len-shapid_len'] == 0]

In [52]:
# Get the maxmun common len in each shape_id
merge_max = merge[merge.groupby(['shape_id'])['common_len'].transform('max') == merge['common_len']]
merge_max.head(5)

,shape_id,StopID_set_x,StopID_list_x,route_short,JourneyPatternID,StopID_set_y,StopID_list_y,common_len,shape_id_len,JourneyPatternID_len,JPID_len-shapid_len
7,0-1-y12-1.2.O,"{0271, 0265, 0340, 0354, 2804, 0378, 0355, 035...","[0265, 0271, 0340, 0350, 0351, 0352, 0353, 035...",0001,00010003,"{0205, 0050, 0271, 0265, 1641, 0229, 0045, 443...","[0045, 0046, 0047, 0048, 0049, 0050, 0051, 005...",2,21,21,0
15,0-1-y12-1.4.I,"{0205, 0223, 0010, 0319, 0017, 0384, 0085, 020...","[0010, 0012, 0014, 0015, 0017, 0018, 0019, 002...",0001,00010001,"{0050, 0271, 0265, 0340, 1641, 0319, 0229, 035...","[0044, 0045, 0046, 0047, 0048, 0049, 0050, 005...",5,46,46,0
25,0-102-y12-1.85.O,"{4331, 3705, 1040, 3634, 3613, 0940, 3641, 438...","[0733, 0788, 0815, 0816, 0905, 0913, 0914, 093...",0102,01020001,"{4331, 3705, 1040, 3634, 3613, 0940, 3641, 438...","[0733, 0788, 0815, 0816, 0905, 0913, 0914, 093...",52,52,52,0
34,0-104-y12-1.88.O,"{0251, 1651, 0223, 0250, 1197, 4791, 4784, 022...","[0218, 0219, 0220, 0221, 0222, 0223, 0224, 022...",0104,01040001,"{0251, 1651, 0223, 0250, 1197, 4791, 4784, 022...","[0218, 0219, 0220, 0221, 0222, 0223, 0224, 022...",32,32,32,0
79,0-120-y12-1.101.I,"{0408, 0829, 0410, 0284, 0079, 0409, 0414, 006...","[0030, 0068, 0069, 0075, 0079, 0080, 0082, 026...",0120,01201002,"{0408, 0829, 0410, 0284, 0079, 0409, 0414, 006...","[0030, 0068, 0069, 0075, 0079, 0080, 0082, 026...",35,35,35,0


In [53]:
# Now we get the JourneyPatternID to its possible shape_id
# Here could have duplicate JPID but it's the same StopID_list so doesn't matter map to which shape_id, they are the same
map_JPId_shapeID = merge_max.loc[:, ['route_short', 'JourneyPatternID', 'shape_id', 'StopID_list_x']]
map_JPId_shapeID.head(5)

,route_short,JourneyPatternID,shape_id,StopID_list_x
7,0001,00010003,0-1-y12-1.2.O,"[0265, 0271, 0340, 0350, 0351, 0352, 0353, 035..."
15,0001,00010001,0-1-y12-1.4.I,"[0010, 0012, 0014, 0015, 0017, 0018, 0019, 002..."
25,0102,01020001,0-102-y12-1.85.O,"[0733, 0788, 0815, 0816, 0905, 0913, 0914, 093..."
34,0104,01040001,0-104-y12-1.88.O,"[0218, 0219, 0220, 0221, 0222, 0223, 0224, 022..."
79,0120,01201002,0-120-y12-1.101.I,"[0030, 0068, 0069, 0075, 0079, 0080, 0082, 026..."


In [54]:
# Export to csv
#map_JPId_shapeID.to_csv('map_JPID_to_shape_id.csv', index=False)

In [55]:
res.shape

(70675418, 11)

#### Step 4. Now delete those rows that StopID should not in that JourneyPatternID

In [56]:
# Base on the mapping made before we can know what StopID should on that routes
# Before we already get the shape_id to its StopID sequence, so we use this as basic
#transit_shapeID_stopID
# Add JourneyPatternID column
map_JPID_to_stopID = pd.merge(transit_shapeID_stopID, map_JPId_shapeID, on='shape_id', how='inner')
map_JPID_to_stopID = map_JPID_to_stopID.loc[:,['JourneyPatternID', 'StopID']]
map_JPID_to_stopID

,JourneyPatternID,StopID
0,00010003,0265
1,00010003,0271
2,00010003,0340
3,00010003,0350
4,00010003,0351
5,00010003,0352
6,00010003,0353
7,00010003,0354
8,00010003,0355
9,00010003,0356


#### 0720 revised
Split into 2 situation. For those JourneyPatternID certainly can map to certain trip_id, we delete those StopID should not on that JouurneyPatternID. Others, we still use the way to delete outlier to judge if need to be remove.

#### First sub-set: JourneyPatternID can map to certain trip_id

In [57]:
# Get those JPID can map to trip_id
can_map = map_JPID_to_stopID['JourneyPatternID'].unique()
res_sub1 = res[res.JourneyPatternID.isin(can_map)]
res_sub1.shape

(5554681, 11)

In [58]:
# Now start to delete
# merge res and map_JPID_to_stopID

res_sub1 = pd.merge(res_sub1, map_JPID_to_stopID, on=['JourneyPatternID','StopID'], how='inner')
res_sub1.head(10)

,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day,Route
0,2013-01-30 19:03:05,00131003,2013-01-30,3356,-6.280505,53.343040,33408,1998,True,Wednesday,0013
1,2013-01-30 19:03:05,00131003,2013-01-30,3356,-6.280505,53.343040,33408,1998,True,Wednesday,0013
2,2013-01-30 19:03:23,00131003,2013-01-30,3356,-6.279584,53.342880,33408,1998,False,Wednesday,0013
3,2013-01-30 19:03:23,00131003,2013-01-30,3356,-6.279584,53.342880,33408,1998,False,Wednesday,0013
4,2013-01-30 19:03:44,00131003,2013-01-30,3356,-6.279526,53.342869,33408,1998,False,Wednesday,0013
5,2013-01-30 19:03:44,00131003,2013-01-30,3356,-6.279526,53.342869,33408,1998,False,Wednesday,0013
6,2013-01-30 19:04:04,00131003,2013-01-30,3356,-6.277749,53.342812,33408,1998,False,Wednesday,0013
7,2013-01-30 19:04:04,00131003,2013-01-30,3356,-6.277749,53.342812,33408,1998,False,Wednesday,0013
8,2013-01-30 19:04:25,00131003,2013-01-30,3356,-6.275193,53.343319,33408,1998,False,Wednesday,0013
9,2013-01-30 19:04:25,00131003,2013-01-30,3356,-6.275193,53.343319,33408,1998,False,Wednesday,0013


In [59]:
res_sub1.shape

(5069620, 11)

#### Second sub-set: If cannot map to any JournayPatternID
In version 3, use the outlier mean - 1.5*IQR to delete the outlier. In this version, for those JourneyPatternID not sure map to which trip_id we use "mean - 1.5*IQR" to delete the outlier.

In [60]:
res_sub2 = res[~res.JourneyPatternID.isin(can_map)]
res_sub2.shape

(65120737, 11)

In [61]:
# Count how many times that StopID occurs in the certain JourneyPatternID

gb = res_sub2.groupby(['JourneyPatternID', 'StopID'])
idx = pd.DataFrame(gb['AtStop'].count())

In [62]:
# Set index back to data

idx.reset_index(['JourneyPatternID', 'StopID'], inplace=True)

# Rename column

idx = idx.rename(columns = {'AtStop':'AtStopCount'})

In [63]:
# In order to find the outlier, here calculate the mean and IQR of that JourneyPatternID + StopID

gb2 = idx.groupby(['JourneyPatternID'])
gb2count =  gb2['AtStopCount'].mean()
gb2q =  gb2['AtStopCount'].quantile(.75) - gb2['AtStopCount'].quantile(.25)
idx2 = pd.concat([pd.DataFrame(gb2count),pd.DataFrame(gb2q)], axis=1)

# In order to merge with idx, here set JourneyPatternID back to data

idx2.reset_index(['JourneyPatternID'], inplace=True)
idx2.columns = ['JourneyPatternID', 'AtStopCountMean', 'AtStopCountIQR']

In [64]:
# Calculate mean - 2 * IQR

idx2['AtStopCountLowerBound'] = idx2['AtStopCountMean'] - 2* idx2['AtStopCountIQR']

In [65]:
# Compare the time that StopID occurs of its JourneyPatternID with the lowerbound

merge = pd.merge(idx, idx2, on=['JourneyPatternID'], how='outer')

# merge will only keep those JourneyPatternID + StopID is outlier

merge = merge[merge['AtStopCount'] < merge['AtStopCountLowerBound']]

# Clean up..

merge.drop(['AtStopCount', 'AtStopCountMean', 'AtStopCountIQR'], axis=1, inplace=True)

In [66]:
# Merge with res, then those AtStopCountLowerBound is null should be kept

merge2 = pd.merge(res_sub2, merge, on=['JourneyPatternID', 'StopID'], how='outer',suffixes='OA')
res_sub2 = merge2[merge2['AtStopCountLowerBound'].isnull()]

# Clean up

res_sub2.drop('AtStopCountLowerBound', axis=1, inplace=True)
del merge, merge2

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [67]:
# Concat two files together

res = pd.concat([res_sub1, res_sub2], axis=0)
res.head(5)

,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day,Route
0,2013-01-30 19:03:05,00131003,2013-01-30,3356,-6.280505,53.343040,33408,1998,True,Wednesday,0013
1,2013-01-30 19:03:05,00131003,2013-01-30,3356,-6.280505,53.343040,33408,1998,True,Wednesday,0013
2,2013-01-30 19:03:23,00131003,2013-01-30,3356,-6.279584,53.342880,33408,1998,False,Wednesday,0013
3,2013-01-30 19:03:23,00131003,2013-01-30,3356,-6.279584,53.342880,33408,1998,False,Wednesday,0013
4,2013-01-30 19:03:44,00131003,2013-01-30,3356,-6.279526,53.342869,33408,1998,False,Wednesday,0013


In [68]:
res.reset_index(inplace=True)
res.to_feather('stage_two_clean_part_one.feather')

In [72]:
res.shape

(70177566, 12)

### Part III. 

#### 1. Delete those rows of same TimeFrame + JourneyPatternID  +  VehicleJourneyID which unique StopID under n.
* 0711: Delete those rows of same TimeFrame + JourneyPatternID + VehicleJourneyID + VehicleID occurance under n times. test n = 10
* 0806 revised:TimeFrame + JourneyPatternID  +  VehicleJourneyID which unique StopID under n. n = 5

In [73]:
gb = res.groupby(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'])

In [74]:
# Delete those count is less than 10

gbcount = gb.StopID.nunique()
idx = pd.DataFrame(gbcount[gbcount < 5])

In [75]:
# Set the index back to data

idx.reset_index(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'], inplace=True)

In [76]:
# Merge back to res to see which one should be keep

merge = pd.merge(res, idx, on=['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'], how='outer',suffixes='oa')

# If null means that row should keep

res = merge[merge['StopIDa'].isnull()]

# Clean up

res.drop('StopIDa', axis=1, inplace=True)
res = res.rename(columns = {'StopIDo':'StopID'})
del merge

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


#### 2. Delete those rows of same TimeFrame + VehicleJourneyID  but exist over one JourneyPatternID

In [77]:
# Due to make following function work need to change datatype here

res["JourneyPatternID"] = res["JourneyPatternID"].astype("object")

# Check how many unique JourneyPatternID under that group (if over 1 is abnormal)

gb = res.groupby(['TimeFrame', 'VehicleJourneyID'])
idx = pd.DataFrame(gb['JourneyPatternID'].unique())
idx['JourneyPatternID'] = idx['JourneyPatternID'].apply(lambda x: len(x))

In [78]:
# Get those rows which unique JourneyPatternID are over 1

idx = idx[idx.JourneyPatternID > 1]

# Set the index back to data

idx.reset_index(['TimeFrame', 'VehicleJourneyID'], inplace=True)


In [79]:
# Merge with res then can see if is null at the column JourneyPatternIDA then should keep
# suffixes='OA' : is set column name to JourneyPatternIDO and JourneyPatternIDA

merge = pd.merge(res, idx, on=['TimeFrame', 'VehicleJourneyID'], how='outer',suffixes='OA')

# Keep those JourneyPatternIDA is null

res = merge[merge['JourneyPatternIDA'].isnull()]

# Clean up

res.drop('JourneyPatternIDA', axis=1, inplace=True)
res = res.rename(columns = {'JourneyPatternIDO':'JourneyPatternID'})
del merge

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [80]:
res.shape

(69539231, 12)

In [81]:
stop here

SyntaxError: invalid syntax (<ipython-input-81-f9ef205a8f26>, line 1)

In [ ]:
# Store as feather

#res = res.reset_index()
#res.to_feather('DBus_stage_three_clean_v1_step1.feather')

### Part III. Add Weather Information

In [ ]:
#res = pd.read_feather('DBus_stage_three_clean_v1_step2.feather')
#res.head(5)

In [82]:
# Read weather data
ch = pd.read_csv('Casement_Hourly_Nov12Jan13_WindRainOnly.csv')
dh = pd.read_csv('DubAirport_Hourly_Nov12Jan13_WindRainOnly.csv')
ph = pd.read_csv('PhoenixPark_Hourly_Nov12Jan13_RainOnly.csv')

In [83]:
dh.dtypes

date     object
rain    float64
wdsp      int64
dtype: object

In [84]:
from datetime import datetime
ch['DateTime'] = ch['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M'))
ch['DateTime'] = ch['DateTime'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))

dh['DateTime'] = dh['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M'))
dh['DateTime'] = dh['DateTime'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))


ph['DateTime'] = ph['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M'))
ph['DateTime'] = ph['DateTime'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))

In [85]:
ch.drop('date', axis=1, inplace=True)
dh.drop('date', axis=1, inplace=True)
ph.drop('date', axis=1, inplace=True)

In [86]:
# Merge
merge = pd.merge(ch, dh, on='DateTime', how='outer')
merge = pd.merge(merge, ph, on='DateTime', how='outer')
merge.head(5)

,rain_x,wdsp_x,DateTime,rain_y,wdsp_y,rain
0,0.0,8,2012-11-06 00,0.0,10,0
1,0.0,8,2012-11-06 01,0.0,10,0
2,0.0,10,2012-11-06 02,0.0,10,0
3,0.0,11,2012-11-06 03,0.0,13,0
4,0.0,10,2012-11-06 04,0.0,12,0


In [87]:
merge.isnull().sum()

rain_x      0
wdsp_x      0
DateTime    0
rain_y      0
wdsp_y      0
rain        0
dtype: int64

In [88]:
# Convert to float
merge['rain'] = merge['rain'].convert_objects(convert_numeric=True)
merge.dtypes

/Users/EmmaChang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


rain_x      float64
wdsp_x        int64
DateTime     object
rain_y      float64
wdsp_y        int64
rain        float64
dtype: object

In [89]:
# Calculate average
merge['Rain_Avg'] = merge.loc[:,['rain_x', 'rain_y', 'rain']].mean(axis=1)
merge['Wind_Speed_Avg'] = merge.loc[:,['wdsp_x', 'wdsp_y']].mean(axis=1)
weather = merge.loc[:,['DateTime', 'Rain_Avg', 'Wind_Speed_Avg']]
weather.head(10)

,DateTime,Rain_Avg,Wind_Speed_Avg
0,2012-11-06 00,0.000000,9.0
1,2012-11-06 01,0.000000,9.0
2,2012-11-06 02,0.000000,10.0
3,2012-11-06 03,0.000000,12.0
4,2012-11-06 04,0.000000,11.0
5,2012-11-06 05,0.000000,11.5
6,2012-11-06 06,0.000000,12.0
7,2012-11-06 07,0.033333,14.5
8,2012-11-06 08,0.000000,15.0
9,2012-11-06 09,0.000000,15.5


In [90]:
# Merge with gps data
res['DateTime'] = res['Timestamp'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))
res_2 = pd.merge(res, weather, on='DateTime', how='left')
res_2.head(5)

,index,Timestamp,JourneyPatternID,TimeFrame,VehicleJourneyID,Lon,Lat,VehicleID,StopID,AtStop,Day,Route,DateTime,Rain_Avg,Wind_Speed_Avg
0,0,2013-01-30 19:03:05,00131003,2013-01-30,3356,-6.280505,53.343040,33408,1998,True,Wednesday,0013,2013-01-30 19,0.0,14.0
1,1,2013-01-30 19:03:05,00131003,2013-01-30,3356,-6.280505,53.343040,33408,1998,True,Wednesday,0013,2013-01-30 19,0.0,14.0
2,2,2013-01-30 19:03:23,00131003,2013-01-30,3356,-6.279584,53.342880,33408,1998,False,Wednesday,0013,2013-01-30 19,0.0,14.0
3,3,2013-01-30 19:03:23,00131003,2013-01-30,3356,-6.279584,53.342880,33408,1998,False,Wednesday,0013,2013-01-30 19,0.0,14.0
4,4,2013-01-30 19:03:44,00131003,2013-01-30,3356,-6.279526,53.342869,33408,1998,False,Wednesday,0013,2013-01-30 19,0.0,14.0


In [95]:
del res

NameError: name 'res' is not defined

In [94]:
res_2.shape

(69539231, 16)

### Part III. Add School Holiday

In [96]:
school_holiday = ['2013-01-01','2013-01-02', '2013-01-03','2013-01-04','2013-01-05','2013-01-06']
school_holiday_days = ['Saturday','Sunday']
res_2['SchoolHoliday'] = np.zeros(res_2.shape[0], dtype=bool)

In [97]:
res_2.loc[res_2.TimeFrame.isin(school_holiday), 'SchoolHoliday'] = True
res_2.loc[res_2.Day.isin(school_holiday_days), 'SchoolHoliday'] = True

In [98]:
res_2.drop('DateTime', axis=1, inplace=True)

In [99]:
res_2.reset_index(i)
res_2.to_feather('DBus_stage_two_clean_v4.feather')

ArrowIOError: IOError: Error writing bytes to file

In [ ]:
res_2.shape

### Part IV. Add Feature: Number of Stops On Route (JPID_length)

In [4]:
#res = pd.read_feather('stage_two_clean_part_one.feather')
#res.drop('index', axis=1, inplace=True)
#res.head(3)

In [ ]:
res = res_2
del res_2

In [13]:
# Groupby TimeFrame + JourneyPatternID + VehicleJourneyID, count the number of unique stops on the journey
gb = res.groupby(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'])
gbcount = gb.StopID.nunique()

In [16]:
idx = gbcount.reset_index(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID'])
idx.rename(columns = {'StopID':'JPID_length'}, inplace=True)
maxCount = idx[idx.groupby(['JourneyPatternID'])['JPID_length'].transform('max') == idx['JPID_length']]
maxCount = maxCount.loc[:,['JourneyPatternID', 'JPID_length']]
maxCount.drop_duplicates(inplace=True)

In [ ]:
# Merge with res
res = pd.merge(res, maxCount, on=['JourneyPatternID'], how='inner')

### Add Feature : Is XBuses or not

In [ ]:
res['XBuses'] = res['JourneyPatternID'].apply(lambda x: '1' if x.find('X') != -1 else '0')

In [ ]:
res.rename(columns = {'CountStopID':'JPID_length'}, inplace=True)

In [ ]:
res.head(2)

In [ ]:
stop here

In [ ]:
# Output to feather
res.drop('index', axis=1, inplace=True)
res.reset_index(inplace=True)
res.to_feather('DBus_stage_two_clean_final.feather')

### Create an input to add the feature JPID_journeys

In [ ]:
res.sort_values(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID', 'Timestamp'], inplace=True)

In [ ]:
gb = res.groupby(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID', 'StopID'])

In [ ]:
# Only keep first row
first = gb.Timestamp.first()

In [ ]:
first = first.reset_index(['TimeFrame', 'JourneyPatternID', 'VehicleJourneyID', 'StopID'])
first.drop('Timestamp', axis=1, inplace=True)
first.head(4)

In [ ]:
# In order to get the SSID, move down the orginal dataframe one step.
first_2 = first.loc[1:]

# In order to concat move down dataframe to original one, need the number of rows the same as original one.
# So fill in one row to last row.
first_last = first.loc[16663374:]
first_move_down = pd.concat([first_2, first_last], axis=0)

In [ ]:
# Rename column
first_move_down.columns = pd.Series(['TimeFrame_m', 'JourneyPatternID_m', 'VehicleJourneyID_m', 'StopID_m'])
first_move_down.reset_index(inplace=True, drop=True)

In [ ]:
# Now concat two dataframe together
get_ssid = pd.concat([first, first_move_down], axis=1)

In [ ]:
get_ssid.head(3)

In [ ]:
# In order to make sure there are the same JPID, use boolean to check
get_ssid['SameJPID'] = np.where(get_ssid['JourneyPatternID'] == get_ssid['JourneyPatternID_m'], True, False)
get_ssid.head(3)

In [ ]:
# Get the SSID of each group
get_ssid['SSID'] = np.where(get_ssid['SameJPID'], get_ssid['StopID'] + get_ssid['StopID_m'], np.nan)
get_ssid

In [ ]:
# 
gb = get_ssid.groupby(['JourneyPatternID', 'SSID'])
count = gb.count()

In [ ]:
# Get JPID to its SSID
JPID_SSID = count[count.StopID > 0]
JPID_SSID.reset_index(['JourneyPatternID', 'SSID'], inplace=True)
JPID_SSID = JPID_SSID.loc[:, ['JourneyPatternID', 'SSID', 'StopID']]
JPID_SSID

In [ ]:
# Get JPID_journeys of each SSID
gb = JPID_SSID.groupby('SSID')
gbcount = gb.JourneyPatternID.nunique()
gbcount = gbcount.reset_index('SSID')


In [ ]:
#
gbcount.rename(columns = {'JourneyPatternID':'JPID_journeys'}, inplace=True)
gbcount.to_csv('input4_JPID_journeys.csv', index=False)